In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
traindf = pd.read_csv('train_label.csv', dtype=str)

def append_ext(fn):
    return fn+".png"

traindf["ID"]=traindf["ID"].apply(append_ext)

In [3]:
train_batchsize = 100
val_batchsize = 10

# datagen = ImageDataGenerator()
datagen = ImageDataGenerator(validation_split=0.25)
        
train_generator = datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "train_image/",
    x_col = "ID",
    y_col = "Label",
    subset = "training",
    batch_size = train_batchsize,
#     seed = 23,
#     shuffle = True,
    class_mode = "categorical",
    target_size = (512,512)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "train_image/",
    x_col = "ID",
    y_col = "Label",
    subset = "validation",
    batch_size = val_batchsize,
#     seed = 23,
#     shuffle = True,
    class_mode = "categorical",
    target_size = (512,512)
)

Found 873 validated image filenames belonging to 3 classes.
Found 291 validated image filenames belonging to 3 classes.


In [4]:
test_generator = datagen.flow_from_directory(
    "test_image", 
#     batch_size = batch_size, 
    class_mode=None, 
    shuffle=False,
    target_size = (512,512)
)

Found 292 images belonging to 1 classes.


In [5]:
from keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

input_tensor = Input(shape=(512, 512, 3))

base_model = VGG19(include_top = False,
    weights="imagenet",
#     input_tensor = input_tensor,
    input_shape = (512, 512, 3),
#     pooling = None,
#     classes = 2,
    classifier_activation = "softmax")

In [6]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 16, 16, 512)       20024384  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              134218752 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 154,246,211
Trainable params: 134,221,827
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# train the model on the new data for a few epochs
history = model.fit_generator(steps_per_epoch = train_generator.samples // train_generator.batch_size,
                           generator = train_generator, 
                           validation_data = valid_generator, 
                           validation_steps = valid_generator.samples // valid_generator.batch_size,
                           epochs = 50)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
8/8 [==============================] - 497s 62s/step - loss: 611.5194 - accuracy: 0.6300 - val_loss: 15.0213 - val_accuracy: 0.8655
Epoch 2/50
8/8 [==============================] - 453s 57s/step - loss: 21.1385 - accuracy: 0.8525 - val_loss: 10.4537 - val_accuracy: 0.8759
Epoch 3/50
8/8 [==============================] - 478s 60s/step - loss: 2.3244 - accuracy: 0.9521 - val_loss: 7.2454 - val_accuracy: 0.8931
Epoch 4/50
8/8 [==============================] - 487s 61s/step - loss: 7.6796 - accuracy: 0.9004 - val_loss: 14.5420 - val_accuracy: 0.8379
Epoch 5/50
8/8 [==============================] - 477s 60s/step - loss: 1.1196 - accuracy: 0.9754 - val_loss: 4.7108 - val_accuracy: 0.9103
Epoch 6/50
8/8 [==============================] - 434s 54s/step - loss: 0.3778 - accuracy: 0.9793 - val_loss: 2.9399 - val_accuracy: 0.9310
Epoch 7/50
8/8 [==============================] - 442s 55s/step - loss: 0.9976

In [ ]:
# Utility function for plotting of the model results
def visualize_results(history):
    # Plot the accuracy and loss curves
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()


# Run the function to illustrate accuracy and loss
visualize_results(history)

In [ ]:
predicted = model.predict(
    test_generator,
#     batch_size=batch_size,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)

In [ ]:
prediction = predicted.argmax(axis=1)

In [ ]:
np.savetxt("predicted.csv", np.dstack((np.arange(prediction.size),prediction))[0],"%d,%d",header="ID,Label",comments="", )